In [1]:
# --- Helper Functions (Update adjust_quantity_to_step in Cell 1) ---

def format_price_correctly(price, tick_size_str):
     """Formats price string EXACTLY to the tickSize precision using quantize."""
     # No changes needed here, assuming the f-string version is already in Cell 1
     tick_size = Decimal(tick_size_str)
     price_decimal = Decimal(str(price))
     # Determine decimal places based on tickSize exponent
     decimal_places = abs(tick_size.as_tuple().exponent)
     # Create a quantizer for rounding down
     # E.g., if tick_size is 0.01, quantizer is 0.01. If 0.00000100, quantizer is 0.000001
     quantizer = tick_size # Use tick_size directly as the quantizer unit
     # Use ROUND_DOWN as required by Binance for prices/quantities usually
     adjusted_price_decimal = price_decimal.quantize(quantizer, rounding=ROUND_DOWN)
     # Format to fixed decimal places, avoiding scientific notation
     return f"{adjusted_price_decimal:.{decimal_places}f}"

# --- THIS IS THE CORRECTED FUNCTION ---
def adjust_quantity_to_step(quantity, step_size_str):
    """
    Adjusts quantity down to the nearest valid step size and returns a
    formatted string suitable for the Binance API, avoiding scientific notation.
    """
    step_size = Decimal(step_size_str)
    quantity_decimal = Decimal(str(quantity))

    # Calculate adjusted quantity using Decimal floor division
    adjusted_quantity_decimal = (quantity_decimal // step_size) * step_size

    # Determine the number of decimal places required from step_size_str
    # Handles cases like "1", "0.1", "0.00001" correctly
    if '.' in step_size_str:
        # Count digits after the decimal point
        decimal_places = len(step_size_str.split('.')[-1])
    else:
        # No decimal point, so 0 decimal places
        decimal_places = 0

    # Format the Decimal result to a string with the correct fixed precision
    # Using '.{decimal_places}f' ensures standard decimal notation (not scientific)
    formatted_quantity_str = f"{adjusted_quantity_decimal:.{decimal_places}f}"

    return formatted_quantity_str
# --- END OF CORRECTED FUNCTION ---

def get_current_balances(api_client):
    # Assume this function from Cell 1 is correct
    """
    Fetches account balances from Binance.US and returns non-zero balances
    as a Pandas DataFrame.
    (Copied from previous notebook)
    """
    if not api_client:
        print("API Client is not available for get_current_balances.")
        return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'))

    try:
        account_info = api_client.get_account()
        balances_raw = account_info.get('balances', [])

        processed_balances = []
        for asset_info in balances_raw:
            free = float(asset_info['free'])
            locked = float(asset_info['locked'])
            if free > 0 or locked > 0:
                processed_balances.append({
                    'Asset': asset_info['asset'],
                    'Free': free,
                    'Locked': locked
                })

        if not processed_balances:
            print("No assets with non-zero balance found.")
            return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'))

        balances_df = pd.DataFrame(processed_balances)
        balances_df.set_index('Asset', inplace=True)
        return balances_df

    except Exception as e:
        print(f"Error fetching account balances: {e}")
        return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'))

def fetch_and_process_klines(api_client, symbol, interval, start_date_str, end_date_str=None):
    # Assume this function from Cell 1 is correct
    """
    Fetches historical klines for a symbol/interval/date range, processes
    them into a pandas DataFrame, and handles potential API limits.
    """
    if not api_client: print("API Client not available for fetch_and_process_klines."); return None
    print(f"Fetching klines for {symbol}, interval {interval}, starting {start_date_str}...")
    try:
        klines_generator = api_client.get_historical_klines_generator(
            symbol, interval, start_date_str, end_str=end_date_str )
        all_klines = list(klines_generator)
        if not all_klines: print(f"No kline data found for {symbol}."); return pd.DataFrame()
        print(f"Fetched {len(all_klines)} klines.")
        columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time',
                   'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
                   'Taker Buy Quote Asset Volume', 'Ignore']
        klines_df = pd.DataFrame(all_klines, columns=columns)
        klines_df['Open Time'] = pd.to_datetime(klines_df['Open Time'], unit='ms', utc=True)
        klines_df['Close Time'] = pd.to_datetime(klines_df['Close Time'], unit='ms', utc=True)
        numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume',
                        'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume']
        for col in numeric_cols: klines_df[col] = pd.to_numeric(klines_df[col])
        klines_df.drop('Ignore', axis=1, inplace=True)
        klines_df.set_index('Open Time', inplace=True)
        print(f"Processed klines for {symbol} ({interval}).")
        return klines_df
    except Exception as e: print(f"Error fetching/processing {symbol} ({interval}): {e}"); return None

# --- Also include necessary imports in the corrected Cell 1 ---
import pandas as pd
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN, getcontext # Make sure getcontext is imported if you use it elsewhere, otherwise optional here
import json
import math
from datetime import datetime, timedelta

print("--- Cell 1: Helper Functions (with corrected adjust_quantity_to_step) Defined ---")
# You would still have the client initialization logic below this in the actual Cell 1

--- Cell 1: Helper Functions (with corrected adjust_quantity_to_step) Defined ---


In [34]:
# Cell 2: Get Comprehensive Portfolio State (Balances & Open Orders) - CORRECTED DISPLAY v2

import pandas as pd
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN # Use Decimal for precision
from datetime import datetime
import warnings

# Suppress specific Pandas warnings if desired, use cautiously
# warnings.simplefilter(action='ignore', category=FutureWarning)

# --- Configuration ---
QUOTE_ASSET = 'USDT' # Define your quote asset
DISPLAY_PRECISION = 8 # How many decimal places to show

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client is not initialized. Please run Cell 1 first.")
if 'get_current_balances_detailed' not in locals() or 'get_open_orders_detailed' not in locals():
     # Define functions locally if Cell 1 wasn't run (not ideal but for completeness)
    print("⚠️ Defining state-fetching functions locally. Run Cell 1 for canonical versions.")
    def get_current_balances_detailed(api_client):
        if not api_client: return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)
        try:
            account_info = api_client.get_account(); balances_raw = account_info.get('balances', [])
            processed = [{'Asset':i['asset'],'Free':Decimal(i['free']),'Locked':Decimal(i['locked'])} for i in balances_raw if Decimal(i['free'])>0 or Decimal(i['locked'])>0]
            if not processed: return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)
            df = pd.DataFrame(processed); df.set_index('Asset', inplace=True); return df
        except Exception as e: print(f"❌ Error balances: {e}"); return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)
    def get_open_orders_detailed(api_client):
        if not api_client: return pd.DataFrame()
        try:
            orders = api_client.get_open_orders()
            if not orders: return pd.DataFrame()
            df = pd.DataFrame(orders)
            for col in ['price','origQty','executedQty','cummulativeQuoteQty','stopPrice']:
                if col in df.columns: df[col] = df[col].apply(lambda x: Decimal(str(x)) if x is not None else Decimal('0'))
            for col in ['time','updateTime']:
                if col in df.columns: df[col] = pd.to_datetime(df[col], unit='ms', utc=True, errors='coerce')
            return df.sort_values(by=['symbol', 'time'], ascending=[True, False]).reset_index(drop=True)
        except Exception as e: print(f"❌ Error orders: {e}"); return pd.DataFrame()

# --- Main Execution Logic ---
print("--- Fetching Comprehensive Portfolio State ---")
portfolio_state = {
    "balances": pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object),
    "open_orders": pd.DataFrame()
}

portfolio_state["balances"] = get_current_balances_detailed(client)
portfolio_state["open_orders"] = get_open_orders_detailed(client)

print("\n--- Portfolio State Summary ---")

# --- Format Balances DataFrame for Display ---
if not portfolio_state["balances"].empty:
    print("Balances (Free/Locked):")
    # Create a copy for display formatting
    balances_display_df = portfolio_state["balances"].copy()
    # Apply string formatting to Decimal columns
    formatter = f'{{:.{DISPLAY_PRECISION}f}}' # e.g., '{:.8f}'
    balances_display_df['Free'] = balances_display_df['Free'].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)
    balances_display_df['Locked'] = balances_display_df['Locked'].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)
    print(balances_display_df)
else:
    print("Balances: None found or error.")

# --- Format Open Orders DataFrame for Display ---
if not portfolio_state["open_orders"].empty:
    print("\nOpen Orders:")
    # Create a copy for display formatting
    orders_display_df = portfolio_state["open_orders"].copy()
    cols_to_format = ['price', 'origQty', 'executedQty']
    cols_to_display = ['symbol','orderId','side','type','status','price','origQty','executedQty','time']
    cols_to_display = [c for c in cols_to_display if c in orders_display_df.columns]
    formatter = f'{{:.{DISPLAY_PRECISION}f}}'

    for col in cols_to_format:
        if col in orders_display_df.columns:
             orders_display_df[col] = orders_display_df[col].apply(lambda x: formatter.format(x) if isinstance(x, Decimal) else x)

    # Print the formatted copy
    print(orders_display_df[cols_to_display])
else:
    print("Open Orders: None found.")

print("\n✅ State captured in 'portfolio_state' dictionary (with original Decimal types).")
# Note: portfolio_state['balances'] and portfolio_state['open_orders'] still contain the precise Decimal objects.

--- Fetching Comprehensive Portfolio State ---
Fetching account balances...
✅ Balances fetched successfully.
Fetching open orders...
Fetched 5 open order(s).
✅ Open orders fetched and processed successfully.

--- Portfolio State Summary ---
Balances (Free/Locked):
             Free       Locked
Asset                         
BTC    0.00000710   0.00008000
USDT   0.10189178  26.60000000
BUSD   0.14956400   0.00000000
WAVES  0.01000000   0.00000000

Open Orders:
    symbol     orderId  side         type status           price     origQty  \
0  BTCUSDT  1466046736  SELL  LIMIT_MAKER    NEW  91000.00000000  0.00004000   
1  BTCUSDT  1466046691  SELL  LIMIT_MAKER    NEW  87000.00000000  0.00004000   
2  BTCUSDT  1466045855   BUY  LIMIT_MAKER    NEW  50000.00000000  0.00018000   
3  BTCUSDT  1466045747   BUY  LIMIT_MAKER    NEW  64000.00000000  0.00014000   
4  BTCUSDT  1466045383   BUY  LIMIT_MAKER    NEW  72000.00000000  0.00012000   

  executedQty                             time  
0  0.

In [38]:
# Cell 3: Identify Top N Assets by Volume

import pandas as pd
import os
from decimal import Decimal # Use Decimal for price/volume if needed later

# --- Configuration ---
N = 10 # Number of top crypto assets to target
QUOTE_ASSET_FILTER = 'USDT' # Should match QUOTE_ASSET usually
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD', 'TUSD', 'USDP', 'FDUSD'] # Define stablecoins to exclude

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client is not initialized. Please run Cell 1 first.")

# --- Initialize ---
top_n_assets = []
top_n_symbols = []
volume_ranking_df = pd.DataFrame() # Will store detailed volume info

# --- Main Logic ---
print(f"\n--- Identifying Top {N} Assets by 24hr {QUOTE_ASSET_FILTER} Volume ---")
try:
    all_tickers = client.get_ticker() # Fetch all symbol tickers
    # Filter for pairs ending with the quote asset (e.g., 'USDT')
    usdt_tickers = [t for t in all_tickers if t.get('symbol', '').endswith(QUOTE_ASSET_FILTER)]
    print(f"Processing {len(usdt_tickers)} {QUOTE_ASSET_FILTER} pairs...")

    volume_data = []
    for ticker in usdt_tickers:
        symbol = ticker.get('symbol')
        if not symbol: continue # Skip if symbol key is missing

        # Derive base asset (e.g., 'BTC' from 'BTCUSDT')
        base_asset = symbol[:-len(QUOTE_ASSET_FILTER)] # More robust than replace

        # Skip if the base asset is a known stablecoin
        if base_asset in STABLECOIN_ASSETS: continue

        try:
             # Extract quote volume and last price, convert safely
             volume_str = ticker.get('quoteVolume', '0.0')
             price_str = ticker.get('lastPrice', '0.0')
             volume = Decimal(volume_str)
             last_price = Decimal(price_str)

             # Only include assets with positive volume and price
             if volume > 0 and last_price > 0:
                  volume_data.append({
                      'Asset': base_asset,
                      'Symbol': symbol,
                      f'Volume_{QUOTE_ASSET_FILTER}': volume, # Store as Decimal
                      'LastPrice': last_price # Store as Decimal
                  })
        except Exception as e:
             # Log ticker processing errors if needed, otherwise skip problematic tickers
             # print(f"  Warning: Skipping {symbol} due to data error: {e}")
             pass # Continue to the next ticker

    if not volume_data:
         print("⚠️ Warning: No valid volume data found after filtering.")
    else:
         # Create DataFrame from collected data
         volume_ranking_df = pd.DataFrame(volume_data)
         # Sort by volume (descending)
         volume_ranking_df.sort_values(by=f'Volume_{QUOTE_ASSET_FILTER}', ascending=False, inplace=True)
         # Select the top N rows
         top_n_df = volume_ranking_df.head(N)
         # Extract asset names and symbols
         top_n_assets = top_n_df['Asset'].tolist()
         top_n_symbols = top_n_df['Symbol'].tolist()
         print(f"Top {N} Assets (by USDT Volume): {top_n_assets}")
         # Uncomment below to see the volume details of the top N
         # with pd.option_context('display.float_format', '{:,.2f}'.format): # Format volume nicely
         #    print(top_n_df[['Asset', 'Symbol', f'Volume_{QUOTE_ASSET_FILTER}']])

except Exception as e:
    print(f"❌ An error occurred fetching/processing ticker data: {e}")
    # Reset variables to ensure clean state on error
    top_n_assets, top_n_symbols, volume_ranking_df = [], [], pd.DataFrame()

# --- Final Check ---
if not top_n_symbols:
    print("⚠️ Top N symbols list is empty. Subsequent steps might fail.")
    # Consider raising an error if this is critical
    # raise ValueError("Failed to identify Top N symbols.")
else:
    print(f"✅ Top {len(top_n_symbols)} symbols identified: {top_n_symbols}")
    print(f"   Volume ranking data stored in 'volume_ranking_df'.")


--- Identifying Top 10 Assets by 24hr USDT Volume ---
Processing 188 USDT pairs...
Top 10 Assets (by USDT Volume): ['XRP', 'ETH', 'BTC', 'SOL', 'ADA', 'HBAR', 'DOGE', 'LTC', 'SUI', 'BNB']
✅ Top 10 symbols identified: ['XRPUSDT', 'ETHUSDT', 'BTCUSDT', 'SOLUSDT', 'ADAUSDT', 'HBARUSDT', 'DOGEUSDT', 'LTCUSDT', 'SUIUSDT', 'BNBUSDT']
   Volume ranking data stored in 'volume_ranking_df'.


In [40]:
# Cell 4: Fetch/Load Historical Data for Top N Symbols - CORRECTED Timezone Handling

import pandas as pd
import os
from binance.client import Client # For interval constants
from datetime import datetime, timezone # Import timezone specifically

# --- Configuration ---
INTERVALS_TO_PROCESS = ['1d', '1h', '5m', '1m'] # Timeframes needed for analysis
INTERVAL_API_MAP = {
    '1d': Client.KLINE_INTERVAL_1DAY, '1h': Client.KLINE_INTERVAL_1HOUR,
    '5m': Client.KLINE_INTERVAL_5MINUTE, '1m': Client.KLINE_INTERVAL_1MINUTE
}
START_DATES_FETCH = {
    '1d': "90 days ago UTC", '1h': "7 days ago UTC",
    '5m': "3 days ago UTC", '1m': "1 day ago UTC"
}
START_DATES_SAVE_IDS = {
    '1d': "90_days_ago_UTC", '1h': "7_days_ago_UTC",
    '5m': "3_days_ago_UTC", '1m': "1_day_ago_UTC"
}
DATA_DIRECTORY = "data"
FORCE_FETCH_FRESH = False

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client not initialized. Run Cell 1.")
if 'fetch_and_process_klines' not in locals():
     raise RuntimeError("Helper function 'fetch_and_process_klines' not defined. Run Cell 1.")
if 'top_n_symbols' not in locals() or not top_n_symbols:
    raise ValueError("'top_n_symbols' list not found or empty. Run Cell 3.")

# --- Initialization ---
top_n_klines = {}
os.makedirs(DATA_DIRECTORY, exist_ok=True)
fetch_errors = 0
load_errors = 0
save_errors = 0

print(f"\n--- Processing Kline Data for Top {len(top_n_symbols)} Symbols ---")
print(f"Intervals: {INTERVALS_TO_PROCESS}. Will {'FETCH FRESH' if FORCE_FETCH_FRESH else 'LOAD local first'}.")

for symbol in top_n_symbols:
    top_n_klines[symbol] = {}
    symbol_fetch_errors = 0

    for interval_key in INTERVALS_TO_PROCESS:
        interval_value = INTERVAL_API_MAP.get(interval_key)
        if not interval_value: continue

        df = None
        loaded_from_csv = False
        source_msg = ""

        # --- 1. Attempt to Load from CSV ---
        if not FORCE_FETCH_FRESH:
            start_id = START_DATES_SAVE_IDS.get(interval_key)
            if start_id:
                filename = os.path.join(DATA_DIRECTORY, f"{symbol}_{interval_key}_{start_id}.csv")
                if os.path.exists(filename):
                    try:
                        # Read CSV, set index, parse dates
                        df = pd.read_csv(filename, index_col='Open Time', parse_dates=True)

                        # --- CORRECTED Timezone Handling ---
                        if df.index.tz is None:
                            # If no timezone info, assume UTC and localize
                            df.index = df.index.tz_localize('UTC')
                            # print(f"  Localized {filename} to UTC.") # Optional debug log
                        # Check if the existing timezone is NOT equivalent to UTC
                        # This handles both pytz objects and datetime.timezone objects robustly
                        elif df.index.tz != timezone.utc:
                             print(f"  Warning: Timezone for {filename} is {df.index.tz}, attempting conversion to UTC.")
                             try:
                                 # Attempt to convert to UTC if it's something else
                                 df.index = df.index.tz_convert('UTC')
                             except Exception as tz_convert_err:
                                 # Log error if conversion fails unexpectedly
                                 print(f"  ⚠️ Error converting timezone for {filename}: {tz_convert_err}. Skipping file.")
                                 load_errors += 1
                                 df = None # Set df to None to trigger fetch
                        # --- End CORRECTED Timezone Handling ---

                        # If df is still valid after timezone check/conversion
                        if df is not None:
                             loaded_from_csv = True
                             source_msg = f"Loaded {symbol} {interval_key} from CSV."

                    except Exception as e:
                        # Catch other potential loading errors (parsing issues etc.)
                        print(f"  ⚠️ Error loading {filename}: {e}. Will try fetching.")
                        load_errors += 1
                        df = None # Ensure df is None if loading fails

        # --- 2. Fetch Fresh if Not Loaded or if Forcing ---
        if df is None: # Fetch if df is None (due to not existing, load error, or FORCE_FETCH_FRESH)
            start_str = START_DATES_FETCH.get(interval_key, "7 days ago UTC")
            print(f"  Fetching {symbol} {interval_key} data starting {start_str}...")
            df = fetch_and_process_klines(client, symbol, interval_value, start_str)
            source_msg = f"Fetched {symbol} {interval_key} from API."

            # --- 3. Optionally Save Freshly Fetched Data ---
            if df is not None and not df.empty:
                 save_id = START_DATES_SAVE_IDS.get(interval_key)
                 if save_id:
                      save_filename = os.path.join(DATA_DIRECTORY, f"{symbol}_{interval_key}_{save_id}.csv")
                      try:
                           df.to_csv(save_filename)
                           source_msg += " Saved to CSV."
                      except Exception as e:
                           print(f"  ⚠️ Error SAVING {save_filename}: {e}")
                           save_errors += 1
                 else:
                      print(f"  ⚠️ Cannot determine save filename for {interval_key}. Data not saved.")
            elif df is None: # Error during fetch/process
                 symbol_fetch_errors += 1
                 source_msg += " Fetch/Process FAILED."

        # --- 4. Store the final DataFrame ---
        if df is not None and not df.empty:
             top_n_klines[symbol][interval_key] = df
        elif not loaded_from_csv: # Only flag error if fetch failed
             print(f"  ⚠️ No data obtained or processed for {symbol} {interval_key}.")

    # --- End of interval loop ---
    if symbol_fetch_errors > 0:
        fetch_errors += symbol_fetch_errors

# --- End of symbol loop ---

# --- Final Summary ---
if load_errors > 0: print(f"⚠️ Encountered {load_errors} error(s) loading existing CSV data.")
if fetch_errors > 0: print(f"⚠️ Encountered {fetch_errors} error(s) fetching/processing fresh data.")
if save_errors > 0: print(f"⚠️ Encountered {save_errors} error(s) saving fetched data.")

missing_data_symbols = []
for symbol, intervals in top_n_klines.items():
     loaded_intervals = list(intervals.keys())
     if len(loaded_intervals) < len(INTERVALS_TO_PROCESS):
          missing = set(INTERVALS_TO_PROCESS) - set(loaded_intervals)
          missing_data_symbols.append(f"{symbol} (missing: {', '.join(missing)})")
if missing_data_symbols:
     print(f"⚠️ Symbols potentially missing required interval data: {'; '.join(missing_data_symbols)}")

print(f"✅ Kline data processing complete. Data stored in 'top_n_klines' dictionary.")


--- Processing Kline Data for Top 10 Symbols ---
Intervals: ['1d', '1h', '5m', '1m']. Will LOAD local first.
✅ Kline data processing complete. Data stored in 'top_n_klines' dictionary.


In [42]:
# Cell 5: Calculate Technical Indicators for Top N Symbols

import pandas as pd
import numpy as np # Needed for manual calculations

# --- Configuration ---
SMA_PERIODS = [20, 50]      # Periods for Simple Moving Averages
RSI_PERIOD = 14             # Period for Relative Strength Index
MACD_FAST = 12              # Fast EMA period for MACD
MACD_SLOW = 26              # Slow EMA period for MACD
MACD_SIGNAL = 9             # Signal Line EMA period for MACD

# --- Check Prerequisites ---
if 'top_n_klines' not in locals() or not top_n_klines:
    raise ValueError("'top_n_klines' dictionary not found or empty. Run Cell 4 first.")

print("\n--- Calculating Technical Indicators ---")
calculation_errors = 0

# Loop through each symbol in the main dictionary
for symbol, interval_dict in top_n_klines.items():
    symbol_errors = 0
    # Loop through each interval ('1d', '1h', etc.) for the current symbol
    for interval, df in interval_dict.items():
        # Ensure DataFrame is not empty and has the 'Close' column needed
        if df.empty or 'Close' not in df.columns:
            # Silently skip empty DFs or those missing essential data
            continue

        try:
            # --- Calculate SMAs ---
            for period in SMA_PERIODS:
                # Ensure enough data points for the rolling window
                if len(df) >= period:
                    # Use min_periods=period to avoid partial calculations at the start
                    df[f'SMA_{period}'] = df['Close'].rolling(window=period, min_periods=period).mean()
                else:
                    df[f'SMA_{period}'] = np.nan # Assign NaN if not enough data

            # --- Calculate RSI ---
            if len(df) >= RSI_PERIOD + 1: # Need at least period+1 for diff()
                delta = df['Close'].diff()
                gain = delta.where(delta > 0, 0.0)
                loss = -delta.where(delta < 0, 0.0)
                # Use Exponential Moving Average for RSI smoothing
                avg_gain = gain.ewm(com=RSI_PERIOD - 1, min_periods=RSI_PERIOD).mean()
                avg_loss = loss.ewm(com=RSI_PERIOD - 1, min_periods=RSI_PERIOD).mean()

                # Calculate Relative Strength (RS) - handle division by zero
                rs = np.where(avg_loss == 0, np.inf, avg_gain / avg_loss) # Avoid division by zero warning

                # Calculate RSI
                rsi = 100.0 - (100.0 / (1.0 + rs))
                rsi[rs == np.inf] = 100.0 # Set RSI to 100 where avg_loss was 0 (infinite RS)

                # If avg_gain and avg_loss are both 0 (e.g., flat price), RSI is undefined (NaN).
                # Fill initial NaNs, potentially assigning a neutral 50.
                rsi = pd.Series(rsi, index=df.index).fillna(50.0)

                df[f'RSI_{RSI_PERIOD}'] = rsi
            else:
                df[f'RSI_{RSI_PERIOD}'] = np.nan # Assign NaN if not enough data


            # --- Calculate MACD ---
            # Need enough data for slow EMA calculation
            if len(df) >= MACD_SLOW:
                ema_fast = df['Close'].ewm(span=MACD_FAST, adjust=False).mean()
                ema_slow = df['Close'].ewm(span=MACD_SLOW, adjust=False).mean()
                macd_line = ema_fast - ema_slow
                # Need enough data points for signal line calculation on macd_line
                if len(macd_line.dropna()) >= MACD_SIGNAL:
                     signal_line = macd_line.ewm(span=MACD_SIGNAL, adjust=False).mean()
                     histogram = macd_line - signal_line
                else:
                     signal_line = np.nan
                     histogram = np.nan

                df[f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = macd_line
                df[f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = signal_line
                df[f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = histogram
            else:
                 # Assign NaN if not enough data for MACD calculation
                 df[f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = np.nan
                 df[f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = np.nan
                 df[f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}'] = np.nan


            # DataFrames are modified in-place within the dictionary

        except Exception as e:
            print(f"  ❌ Error calculating indicators for {symbol} {interval}: {e}")
            symbol_errors += 1
            # Attempt to remove potentially partially added columns to avoid downstream errors
            potential_new_cols = [f'SMA_{p}' for p in SMA_PERIODS] + \
                                 [f'RSI_{RSI_PERIOD}', f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', \
                                  f'MACDs_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}', f'MACDh_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}']
            for col in potential_new_cols:
                 if col in df.columns:
                      try: df.drop(columns=[col], inplace=True)
                      except Exception: pass # Ignore errors during cleanup


    if symbol_errors > 0:
        calculation_errors += symbol_errors

if calculation_errors > 0:
     print(f"⚠️ Indicator calculation completed with {calculation_errors} total errors.")

# --- Optional: Display sample data to verify ---
# symbol_to_show = top_n_symbols[0] if top_n_symbols else None
# interval_to_show = '1h'
# if symbol_to_show and interval_to_show in top_n_klines.get(symbol_to_show, {}):
#     print(f"\n--- Sample {symbol_to_show} {interval_to_show} Data with Indicators (Last 5 Rows) ---")
#     df_sample = top_n_klines[symbol_to_show][interval_to_show]
#     cols = ['Close', f'SMA_{SMA_PERIODS[0]}', f'SMA_{SMA_PERIODS[1]}',
#             f'RSI_{RSI_PERIOD}', f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}']
#     cols = [c for c in cols if c in df_sample.columns] # Only include columns that exist
#     with pd.option_context('display.float_format', '{:.4f}'.format): # Format display
#         print(df_sample[cols].tail())

print("✅ Indicator calculation complete for all symbols and intervals.")


--- Calculating Technical Indicators ---
✅ Indicator calculation complete for all symbols and intervals.


In [43]:
# Cell 6: Calculate Dynamic Target Allocations

import pandas as pd
import numpy as np
from decimal import Decimal # Use Decimal if dealing with volume/prices from DF

# --- Configuration ---
# Ensure N matches the number of assets selected in Cell 3
TOP_N_ASSETS_COUNT = 10
STABLECOIN_RESERVE_PCT = Decimal('20.0') # Target reserve percentage as Decimal
STABLECOIN_ASSETS = ['USDT', 'USDC', 'BUSD', 'TUSD', 'USDP', 'FDUSD']
# Primary quote/stablecoin, should match filter used in Cell 3
QUOTE_ASSET = 'USDT'

# --- Check Prerequisites ---
if 'volume_ranking_df' not in locals() or volume_ranking_df.empty:
    # Attempt to regenerate volume_ranking_df if it's missing (e.g., error in Cell 3)
    print("⚠️ 'volume_ranking_df' missing, attempting regeneration from tickers...")
    try:
        # Minimal regeneration logic (assumes client exists from Cell 1)
        if 'client' not in locals() or client is None: raise RuntimeError("Client missing for regen")
        all_tickers = client.get_ticker()
        usdt_tickers = [t for t in all_tickers if t.get('symbol', '').endswith(QUOTE_ASSET)]
        volume_data = []
        for ticker in usdt_tickers:
            symbol=ticker.get('symbol'); base=symbol[:-len(QUOTE_ASSET)]; vol_str=ticker.get('quoteVolume','0'); price_str=ticker.get('lastPrice','0')
            if base not in STABLECOIN_ASSETS:
                try: # Convert safely
                    vol=Decimal(vol_str); price=Decimal(price_str)
                    if vol > 0 and price > 0: volume_data.append({'Asset':base,'Symbol':symbol,f'Volume_{QUOTE_ASSET}':vol})
                except Exception: pass # Skip invalid tickers
        if volume_data:
            volume_ranking_df = pd.DataFrame(volume_data)
            volume_ranking_df.sort_values(f'Volume_{QUOTE_ASSET}',ascending=False,inplace=True)
        else:
            volume_ranking_df = pd.DataFrame() # Ensure empty if still no data

        if volume_ranking_df.empty: raise ValueError("Could not regenerate volume data.")
        print("✅ Volume ranking regenerated.")
    except Exception as e:
        # If regeneration fails, targets cannot be calculated dynamically
        raise ValueError(f"Volume data unavailable and regeneration failed: {e}. Cannot calculate targets. Check Cell 3.")

if f'Volume_{QUOTE_ASSET}' not in volume_ranking_df.columns:
     raise KeyError(f"Required volume column 'Volume_{QUOTE_ASSET}' not found in volume_ranking_df. Check Cell 3.")


# --- Initialize ---
# Stores target allocation percentages as Decimals
dynamic_target_allocations_pct = {}

print("\n--- Calculating Dynamic Target Allocations ---")
try:
    # Filter out stablecoins again just in case, and get top N
    crypto_volume_df = volume_ranking_df[~volume_ranking_df['Asset'].isin(STABLECOIN_ASSETS)].copy()
    top_n_crypto_df = crypto_volume_df.head(TOP_N_ASSETS_COUNT)

    # Calculate total volume of the Top N crypto assets (ensure Decimal)
    total_top_n_volume = top_n_crypto_df[f'Volume_{QUOTE_ASSET}'].sum()
    if not isinstance(total_top_n_volume, Decimal):
         total_top_n_volume = Decimal(str(total_top_n_volume)) # Convert if needed

    if total_top_n_volume <= 0:
        print("⚠️ Warning: Total volume for Top N crypto assets is zero or negative. Assigning 0% target weights to crypto.")
        total_top_n_volume = Decimal('0') # Prevent division by zero
    else:
         print(f"Total 24hr Volume for Top {TOP_N_ASSETS_COUNT} crypto assets: {total_top_n_volume:,.2f} {QUOTE_ASSET}")

    # --- Calculate Dynamic Crypto Allocations ---
    # Percentage available for non-stablecoins
    crypto_allocation_pct = Decimal('100.0') - STABLECOIN_RESERVE_PCT

    if total_top_n_volume > 0:
        # Calculate weight based on volume contribution to the Top N total
        for index, row in top_n_crypto_df.iterrows():
            asset = row['Asset']
            # Ensure volume is Decimal
            volume = row[f'Volume_{QUOTE_ASSET}']
            if not isinstance(volume, Decimal): volume = Decimal(str(volume))

            # Calculate target percentage for this asset
            target_pct = (volume / total_top_n_volume) * crypto_allocation_pct
            dynamic_target_allocations_pct[asset] = target_pct
    else:
         # If total volume is zero, assign 0% target to all Top N crypto assets
         for asset in top_n_crypto_df['Asset'].tolist():
              dynamic_target_allocations_pct[asset] = Decimal('0.0')


    # --- Add Stablecoin Allocation ---
    # Assign the reserve percentage to the primary quote asset
    if QUOTE_ASSET in dynamic_target_allocations_pct:
         # This case should be rare if QUOTE_ASSET is in STABLECOIN_ASSETS and excluded above
         dynamic_target_allocations_pct[QUOTE_ASSET] += STABLECOIN_RESERVE_PCT
         print(f"Warning: Primary quote asset {QUOTE_ASSET} was already in crypto targets?")
    else:
         dynamic_target_allocations_pct[QUOTE_ASSET] = STABLECOIN_RESERVE_PCT

    # --- Ensure all target assets (Top N + Stablecoins) are in the dict ---
    # Create a set of all assets that *should* have a target (even if 0%)
    all_intended_target_assets = set(top_n_crypto_df['Asset'].tolist() + STABLECOIN_ASSETS)
    for asset in all_intended_target_assets:
         if asset not in dynamic_target_allocations_pct:
              dynamic_target_allocations_pct[asset] = Decimal('0.0') # Assign 0% if missing

    # --- Verify Sum ---
    total_dynamic_pct = sum(dynamic_target_allocations_pct.values())
    print(f"Target Allocations Sum: {total_dynamic_pct:.2f}%") # Display with 2 decimal places
    # Use Decimal for comparison threshold
    if abs(total_dynamic_pct - Decimal('100.0')) > Decimal('0.01'):
        print("⚠️ WARNING: Target Sum is not 100%!")

    # --- Optional: Print sorted targets for review ---
    # print("\nCalculated Target Allocations (%):")
    # sorted_targets = dict(sorted(dynamic_target_allocations_pct.items(), key=lambda item: item[1], reverse=True))
    # for asset, pct in sorted_targets.items():
    #     if pct > 0: print(f"  {asset}: {pct:.2f}%") # Show non-zero targets with 2 decimals

except Exception as e:
    print(f"❌ An error occurred during dynamic target calculation: {e}")
    dynamic_target_allocations_pct = {} # Reset on error

if not dynamic_target_allocations_pct:
    print("⚠️ Dynamic target allocation dictionary is empty.")
else:
    print("✅ Dynamic target allocations calculated ('dynamic_target_allocations_pct').")


--- Calculating Dynamic Target Allocations ---
Total 24hr Volume for Top 10 crypto assets: 5,053,850.07 USDT
Target Allocations Sum: 100.00%
✅ Dynamic target allocations calculated ('dynamic_target_allocations_pct').


In [44]:
# Cell 7: Define Support/Resistance Helper Function

import numpy as np
import pandas as pd

# --- Check Prerequisites ---
# This function relies on pandas DataFrames having 'Low' and 'High' columns.
# No external variables needed besides the DataFrame passed in.

print("\n--- Defining Helper Function: find_simple_sr ---")

def find_simple_sr(df, lookback=20):
    """
    Identifies simple recent swing lows (support) and highs (resistance).
    Looks for the min Low and max High over a rolling lookback period.

    Args:
        df (pd.DataFrame): Kline DataFrame with 'Low' and 'High' columns.
        lookback (int): How many recent periods (rows) to consider.

    Returns:
        tuple: (recent_support, recent_resistance) as floats or Decimals
               Returns (None, None) if not enough data or columns missing.
    """
    required_cols = ['Low', 'High']
    if df is None or df.empty or len(df) < lookback or not all(c in df.columns for c in required_cols):
        # print(f"  Debug SR: DataFrame empty, too short, or missing columns. Len={len(df) if df is not None else 'None'}, Lookback={lookback}")
        return None, None # Not enough data or required columns missing

    try:
        # Get the 'Low' and 'High' series for the lookback period
        recent_lows = df['Low'].tail(lookback)
        recent_highs = df['High'].tail(lookback)

        # Find the minimum low (support) and maximum high (resistance)
        support = recent_lows.min()
        resistance = recent_highs.max()

        # Return the values (will be float or Decimal depending on df input type)
        return support, resistance

    except Exception as e:
        print(f"  ❌ Error finding S/R: {e}")
        return None, None

print("✅ Helper function 'find_simple_sr' defined.")
# --- Example Test (Optional - Uncomment to run) ---
# if 'top_n_klines' in locals() and top_n_klines and top_n_symbols:
#     test_symbol = top_n_symbols[0]
#     test_interval = '1d'
#     if test_symbol in top_n_klines and test_interval in top_n_klines[test_symbol]:
#         test_df = top_n_klines[test_symbol][test_interval]
#         sup, res = find_simple_sr(test_df, lookback=30) # Example lookback
#         print(f"\nExample {test_symbol} Daily S/R (lookback 30):")
#         if sup is not None and res is not None:
#             print(f"  Support: {sup:.4f}, Resistance: {res:.4f}")
#             last_close = test_df['Close'].iloc[-1]
#             print(f"  Last Close: {last_close:.4f}")
#             print(f"  Dist to Support: {(last_close - sup):.4f}")
#             print(f"  Dist to Resistance: {(res - last_close):.4f}")
#         else:
#             print("  Could not calculate S/R (check data).")
#     else:
#         print(f"\nCannot run example S/R test: {test_symbol} {test_interval} data missing.")
# else:
#     print("\nCannot run example S/R test: Prerequisite data missing.")
# --- End Example Test ---


--- Defining Helper Function: find_simple_sr ---
✅ Helper function 'find_simple_sr' defined.


In [45]:
# Cell 8: Analyze Market State (TA) for Top N Symbols

import numpy as np
import json
from decimal import Decimal # Import Decimal

# --- Configuration ---
SR_LOOKBACK_DAILY = 30
SR_LOOKBACK_HOURLY = 50
RSI_OVERSOLD_THRESHOLD = Decimal('30.0') # Use Decimal for comparison consistency
NEAR_SUPPORT_PROXIMITY_PCT = Decimal('1.0') # Within 1% *above* hourly support

# --- Check Prerequisites ---
if 'top_n_klines' not in locals() or not top_n_klines:
    raise ValueError("'top_n_klines' dictionary not found or empty. Run Cell 5.")
if 'find_simple_sr' not in locals():
     raise RuntimeError("Helper function 'find_simple_sr' not defined. Run Cell 7.")

print("\n--- Analyzing Market State (TA) for Top N Symbols ---")

def analyze_top_n_state(klines_data_dict, sr_lookback_d=30, sr_lookback_h=50, rsi_period=14, rsi_oversold=Decimal('30.0'), proximity_pct=Decimal('1.0')):
    """
    Performs multi-timeframe technical analysis including S/R for multiple symbols.

    Args:
        klines_data_dict (dict): Nested dict {'SYMBOL': {'interval': DataFrame}}. Kline data should include indicators.
        sr_lookback_d (int): Lookback period for daily S/R.
        sr_lookback_h (int): Lookback period for hourly S/R.
        rsi_period (int): The period used for RSI calculation (for key lookup).
        rsi_oversold (Decimal): The threshold below which RSI is considered oversold.
        proximity_pct (Decimal): The percentage above support to consider 'near'.

    Returns:
        dict: Nested dict {'SYMBOL': {'analysis_key': value}}. Contains analysis results or error messages.
    """
    all_analysis = {}
    analysis_errors = 0

    required_intervals = ['1d', '1h', '5m'] # Intervals needed for this specific analysis logic

    # Define expected indicator column names dynamically
    sma50_col = 'SMA_50'
    sma20_col = 'SMA_20'
    rsi_col = f'RSI_{rsi_period}'
    macd_col = f'MACD_{MACD_FAST}_{MACD_SLOW}_{MACD_SIGNAL}' # Example, adjust if needed elsewhere

    for symbol, interval_dict in klines_data_dict.items():
        analysis = {} # Analysis results for this symbol
        all_analysis[symbol] = analysis # Add symbol's dict to the main results dict

        # --- Check for necessary data & minimum lengths ---
        if not all(k in interval_dict for k in required_intervals):
            analysis['error'] = "Missing Interval Data (1d, 1h, or 5m)"
            analysis_errors += 1
            continue # Skip this symbol if essential intervals are missing

        df_1d = interval_dict['1d']
        df_1h = interval_dict['1h']
        df_5m = interval_dict['5m']

        # Check for required columns in specific dataframes and minimum lengths
        required_cols_1d = ['Close', 'Low', 'High', sma50_col]
        required_cols_1h = ['Close', 'Low', 'High', sma20_col]
        required_cols_5m = ['Close', rsi_col]

        if not all(c in df_1d.columns for c in required_cols_1d) or len(df_1d) < sr_lookback_d:
             analysis['error'] = f"Missing 1d columns ({required_cols_1d}) or insufficient length (<{sr_lookback_d})"
             analysis_errors += 1; continue
        if not all(c in df_1h.columns for c in required_cols_1h) or len(df_1h) < sr_lookback_h:
             analysis['error'] = f"Missing 1h columns ({required_cols_1h}) or insufficient length (<{sr_lookback_h})"
             analysis_errors += 1; continue
        if not all(c in df_5m.columns for c in required_cols_5m) or len(df_5m) < rsi_period + 1: # Need enough for RSI calc
             analysis['error'] = f"Missing 5m columns ({required_cols_5m}) or insufficient length (<{rsi_period+1})"
             analysis_errors += 1; continue

        # --- Get Latest Valid Close Price ---
        # Use hourly close for most recent price, ensure it's not NaN
        try:
            last_close = df_1h['Close'].iloc[-1]
            if pd.isna(last_close): raise ValueError("Last hourly close is NaN")
            analysis['last_close'] = Decimal(str(last_close)) # Store as Decimal
        except (IndexError, ValueError) as e:
            analysis['error'] = f"Could not get valid last hourly close: {e}"
            analysis_errors += 1; continue # Cannot proceed without a price

        # --- Perform Analysis Steps ---
        try:
            # 1. Daily Trend (Close vs SMA50) - Check for NaN
            last_daily_close = df_1d['Close'].iloc[-1]
            last_daily_sma50 = df_1d[sma50_col].iloc[-1]
            if pd.isna(last_daily_close) or pd.isna(last_daily_sma50): analysis['daily_trend_up'] = None
            else: analysis['daily_trend_up'] = last_daily_close > last_daily_sma50

            # 2. Hourly Position (Close vs SMA20) - Check for NaN
            last_hourly_sma20 = df_1h[sma20_col].iloc[-1]
            if pd.isna(last_hourly_sma20): analysis['hourly_below_sma'] = None
            else: analysis['hourly_below_sma'] = last_close < last_hourly_sma20 # Use hourly close

            # 3. 5-Minute RSI State - Check for NaN
            last_5m_rsi = df_5m[rsi_col].iloc[-1]
            if pd.isna(last_5m_rsi):
                analysis[f'last_5m_{rsi_col}'] = None # Store actual value as None
                analysis['5m_rsi_oversold'] = None
            else:
                analysis[f'last_5m_{rsi_col}'] = Decimal(str(last_5m_rsi)) # Store actual RSI value as Decimal
                analysis['5m_rsi_oversold'] = analysis[f'last_5m_{rsi_col}'] < rsi_oversold

            # 4. Support/Resistance Check (Using helper from Cell 7)
            daily_sup, daily_res = find_simple_sr(df_1d, lookback=sr_lookback_d)
            hourly_sup, hourly_res = find_simple_sr(df_1h, lookback=sr_lookback_h)
            analysis['daily_support'] = Decimal(str(daily_sup)) if daily_sup is not None else None
            analysis['daily_resistance'] = Decimal(str(daily_res)) if daily_res is not None else None
            analysis['hourly_support'] = Decimal(str(hourly_sup)) if hourly_sup is not None else None
            analysis['hourly_resistance'] = Decimal(str(hourly_res)) if hourly_res is not None else None

            # 5. Proximity to Hourly Support Check
            analysis['near_hourly_support'] = None
            if analysis['last_close'] is not None and analysis['hourly_support'] is not None:
                 lower_bound = analysis['hourly_support']
                 # Calculate upper bound: support * (1 + pct/100)
                 upper_bound = analysis['hourly_support'] * (Decimal('1.0') + proximity_pct / Decimal('100.0'))
                 # Check if last close is between support and support + proximity %
                 analysis['near_hourly_support'] = (lower_bound <= analysis['last_close'] <= upper_bound)

        except IndexError:
             # Handles cases where iloc[-1] fails unexpectedly (should be caught by length checks earlier)
             analysis['error'] = "Not enough data points for indicator/price lookup (IndexError)."
             analysis_errors += 1
        except Exception as e:
             analysis['error'] = f"Unexpected error during analysis: {e}"
             analysis_errors += 1


    if analysis_errors > 0:
         print(f"⚠️ Analysis completed with errors for {analysis_errors} symbol(s).")

    return all_analysis


# --- Run the multi-asset analysis ---
full_market_state = analyze_top_n_state(
    top_n_klines,
    sr_lookback_d=SR_LOOKBACK_DAILY,
    sr_lookback_h=SR_LOOKBACK_HOURLY,
    rsi_period=RSI_PERIOD, # Pass the RSI period used
    rsi_oversold=RSI_OVERSOLD_THRESHOLD,
    proximity_pct=NEAR_SUPPORT_PROXIMITY_PCT
)

print("\n--- Market State Analysis Summary ---")
# Optionally print a compact summary or just confirm completion
analyzed_count = len(full_market_state)
error_count = sum(1 for state in full_market_state.values() if 'error' in state)
success_count = analyzed_count - error_count

print(f"Attempted analysis for {analyzed_count} symbols.")
if error_count > 0:
    print(f"  {error_count} symbols had errors during analysis.")
    # Optionally list symbols with errors:
    # errors = {sym: state['error'] for sym, state in full_market_state.items() if 'error' in state}
    # print(f"  Errors: {errors}")
print(f"  {success_count} symbols analyzed successfully.")
print("\n✅ Analysis results stored in 'full_market_state' dictionary.")

# --- Optional: Display Full State for one symbol ---
# test_symbol_state = 'BTCUSDT'
# if test_symbol_state in full_market_state:
#     print(f"\n--- Detailed State for {test_symbol_state} ---")
#     # Use jsonumps for pretty printing, converting Decimals to strings
#     print(json.dumps(full_market_state[test_symbol_state], indent=4, default=str))
# else:
#      print(f"\nState for {test_symbol_state} not available.")
# --- End Optional Display ---


--- Analyzing Market State (TA) for Top N Symbols ---

--- Market State Analysis Summary ---
Attempted analysis for 10 symbols.
  10 symbols analyzed successfully.

✅ Analysis results stored in 'full_market_state' dictionary.


In [46]:
# Cell 9: Determine Ideal Rebalancing Trades (Based on Total Value Deviation)

import pandas as pd
import numpy as np
from decimal import Decimal, ROUND_DOWN, DivisionByZero # Import Decimal and specific exceptions

# --- Configuration ---
QUOTE_ASSET = 'USDT'
# Minimum trade value threshold to consider a rebalance trade significant
TRADE_THRESHOLD_USDT = Decimal('0.50') # Example: $0.50

# --- Check Prerequisites ---
if 'portfolio_state' not in locals() or 'balances' not in portfolio_state or portfolio_state['balances'].empty:
     raise ValueError("Portfolio balances ('portfolio_state['balances']') not found or empty. Run Cell 2.")
if 'dynamic_target_allocations_pct' not in locals() or not dynamic_target_allocations_pct:
     raise ValueError("Target allocations ('dynamic_target_allocations_pct') not found or empty. Run Cell 6.")
if 'full_market_state' not in locals() or not full_market_state:
     raise ValueError("Market state analysis ('full_market_state') not found or empty. Run Cell 8.")
if 'client' not in locals() or client is None: # Needed for fetching prices if missing
    raise RuntimeError("Binance client is not initialized. Run Cell 1.")


print("\n--- Determining Ideal Rebalancing Trades (Based on Total Value Deviation) ---")

# --- 1. Calculate Current Total Value of Each Asset ---
current_portfolio_details = {}
total_portfolio_value_usdt = Decimal('0.0')

print("Calculating current portfolio value...")
balances_df = portfolio_state['balances'] # From Cell 2

for asset, balance_info in balances_df.iterrows():
    # Total quantity = Free + Locked
    total_quantity = balance_info['Free'] + balance_info['Locked']
    if total_quantity <= 0: continue # Skip zero-balance assets

    current_price = Decimal('0.0')
    usdt_value = Decimal('0.0')
    price_source = "Unknown"

    # Get price: Prioritize analysis state, then fetch if needed
    symbol = f"{asset}{QUOTE_ASSET}"
    if asset == QUOTE_ASSET:
        current_price = Decimal('1.0')
        price_source = "Assumed 1.0"
    elif symbol in full_market_state and 'last_close' in full_market_state[symbol] and full_market_state[symbol]['last_close'] is not None:
        # Use price from TA results (should be Decimal already)
        current_price = full_market_state[symbol]['last_close']
        price_source = "TA Result"
    else:
        # Fallback: Fetch current ticker price if not found in TA or if TA failed for symbol
        try:
            # print(f"  Fetching price for {symbol}...") # Uncomment for debug
            ticker_info = client.get_symbol_ticker(symbol=symbol)
            current_price = Decimal(ticker_info['price'])
            price_source = "API Fetch"
        except Exception as e:
            print(f"  ⚠️ Warning: Could not get price for {symbol}: {e}. Value will be 0.")
            current_price = Decimal('0.0') # Assign 0 if price unavailable
            price_source = "Fetch Failed"

    # Calculate USDT value for the asset
    usdt_value = total_quantity * current_price
    total_portfolio_value_usdt += usdt_value

    current_portfolio_details[asset] = {
        'TotalQuantity': total_quantity,
        'CurrentPrice': current_price,
        'CurrentValueUSDT': usdt_value,
        'PriceSource': price_source
    }

print(f"Total Portfolio Value (Free+Locked): {total_portfolio_value_usdt:,.2f} {QUOTE_ASSET}")

# --- 2. Create Combined DataFrame for Deviation Analysis ---
# Convert details dict to DataFrame
current_values_df = pd.DataFrame.from_dict(current_portfolio_details, orient='index')
# Convert target percentages dict to Series
target_alloc_series = pd.Series(dynamic_target_allocations_pct, name='Target%').apply(Decimal) # Ensure Decimal

# Combine current values and targets
# Use outer join to include assets in targets but not currently held, or vice-versa
combined_df = current_values_df.join(target_alloc_series, how='outer')

# Fill NaNs resulting from the join
combined_df['TotalQuantity'] = combined_df['TotalQuantity'].fillna(Decimal('0.0'))
combined_df['CurrentPrice'] = combined_df['CurrentPrice'].fillna(Decimal('0.0'))
combined_df['CurrentValueUSDT'] = combined_df['CurrentValueUSDT'].fillna(Decimal('0.0'))
combined_df['Target%'] = combined_df['Target%'].fillna(Decimal('0.0'))
combined_df['PriceSource'] = combined_df['PriceSource'].fillna('N/A')

# Recalculate total value from combined DF to ensure consistency after outer join
total_portfolio_value_final = combined_df['CurrentValueUSDT'].sum()

# --- 3. Calculate Allocations and Deviations ---
if total_portfolio_value_final > 0:
    combined_df['CurrentAllocation%'] = (combined_df['CurrentValueUSDT'] / total_portfolio_value_final) * Decimal('100.0')
else:
    combined_df['CurrentAllocation%'] = Decimal('0.0') # Avoid division by zero

combined_df['TargetValueUSDT'] = (combined_df['Target%'] / Decimal('100.0')) * total_portfolio_value_final
combined_df['DeviationValueUSDT'] = combined_df['CurrentValueUSDT'] - combined_df['TargetValueUSDT']
combined_df['Deviation%'] = combined_df['CurrentAllocation%'] - combined_df['Target%']

# --- 4. Determine Initial BUY/SELL Trades based on Deviation Value Threshold ---
proposed_trades_list = []
# Iterate through assets with significant deviation
for asset, row in combined_df.iterrows():
    deviation_value = row['DeviationValueUSDT']
    action = None
    trade_value = Decimal('0.0')
    trade_quantity = Decimal('0.0')

    # Propose SELL if significantly over-allocated
    if deviation_value > TRADE_THRESHOLD_USDT:
        action = 'SELL'
        trade_value = deviation_value # Amount to sell in USDT value
        # Calculate quantity to sell
        current_price = row['CurrentPrice']
        if current_price > 0:
             try:
                 trade_quantity = (trade_value / current_price).quantize(Decimal('1e-12')) # Use high precision intermediate
             except DivisionByZero:
                 print(f"  ⚠️ Warning: Price is zero for {asset}. Cannot calculate sell quantity.")
                 continue # Skip this trade
        else:
             print(f"  ⚠️ Warning: Price is zero for {asset}. Cannot calculate sell quantity.")
             continue # Skip this trade

    # Propose BUY if significantly under-allocated
    elif deviation_value < -TRADE_THRESHOLD_USDT:
        action = 'BUY'
        trade_value = abs(deviation_value) # Amount to buy in USDT value
        # Calculate quantity to buy
        current_price = row['CurrentPrice']
        # Special case for buying stablecoins (use value as quantity)
        if asset == QUOTE_ASSET:
             trade_quantity = trade_value
             current_price = Decimal('1.0') # Assume price 1 for calculation consistency
        elif current_price > 0:
             try:
                 trade_quantity = (trade_value / current_price).quantize(Decimal('1e-12')) # Use high precision intermediate
             except DivisionByZero:
                  print(f"  ⚠️ Warning: Price is zero for {asset}. Cannot calculate buy quantity.")
                  continue # Skip this trade
        else:
             # Try fetching price again if it was missing/zero earlier
             symbol = f"{asset}{QUOTE_ASSET}"
             print(f"  Attempting price re-fetch for BUY {asset}...")
             try:
                 ticker_info = client.get_symbol_ticker(symbol=symbol)
                 current_price = Decimal(ticker_info['price'])
                 if current_price > 0:
                      trade_quantity = (trade_value / current_price).quantize(Decimal('1e-12'))
                      print(f"    Re-fetched price: {current_price}")
                      # Update price in combined_df for reference? Optional.
                      # combined_df.loc[asset, 'CurrentPrice'] = current_price
                 else:
                      print(f"    ⚠️ Re-fetched price is zero for {asset}. Cannot calculate buy quantity.")
                      continue
             except Exception as e:
                  print(f"    ⚠️ Re-fetch failed for {symbol}: {e}. Cannot calculate buy quantity.")
                  continue # Skip this trade


    # Add valid proposed trades to the list
    if action and trade_quantity > 0:
        proposed_trades_list.append({
            'Action': action,
            'Asset': asset,
            # Store intermediate high-precision quantity. Final formatting happens later.
            'IdealQuantity': trade_quantity,
            'IdealValueUSDT': trade_value,
            'CurrentPrice': current_price # Store price used for calculation
        })

# --- 5. Create Final DataFrame of Proposed Trades ---
trades_df = pd.DataFrame() # Initialize empty
if proposed_trades_list:
    trades_df = pd.DataFrame(proposed_trades_list)
    # Sort for clarity (e.g., Sells first, then Buys)
    trades_df.sort_values(by=['Action','Asset'], ascending=[False, True], inplace=True)
    trades_df = trades_df.reset_index(drop=True) # Reset index after sort

    print("\n--- Ideal Rebalancing Trades Proposed (Ignoring Free/Locked Status for now) ---")
    # Display with formatting
    display_cols = ['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT', 'CurrentPrice']
    with pd.option_context('display.float_format', '{:.8f}'.format): # Show more precision for quantity
        print(trades_df[display_cols])

else:
    print("\n--- No significant deviations found requiring ideal rebalancing trades. ---")


# Result: 'trades_df' DataFrame contains the ideal trades based on total value deviation.
print(f"\n✅ Ideal trades calculation complete. Found {len(trades_df)} potential trades.")
print("   Stored results in 'trades_df'. Next step: Check feasibility against free balances.")


--- Determining Ideal Rebalancing Trades (Based on Total Value Deviation) ---
Calculating current portfolio value...
Total Portfolio Value (Free+Locked): 33.67 USDT
  Attempting price re-fetch for BUY ADA...
    Re-fetched price: 0.57730000
  Attempting price re-fetch for BUY DOGE...
    Re-fetched price: 0.14708000
  Attempting price re-fetch for BUY ETH...
    Re-fetched price: 1493.50000000
  Attempting price re-fetch for BUY HBAR...
    Re-fetched price: 0.15410000
  Attempting price re-fetch for BUY LTC...
    Re-fetched price: 70.04000000
  Attempting price re-fetch for BUY SOL...
    Re-fetched price: 104.54000000
  Attempting price re-fetch for BUY XRP...
    Re-fetched price: 1.86560000

--- Ideal Rebalancing Trades Proposed (Ignoring Free/Locked Status for now) ---
  Action Asset    IdealQuantity                  IdealValueUSDT   CurrentPrice
0   SELL   BTC   0.000033521895   2.611459875111559997139675196       77903.11
1   SELL  USDT  19.967891386760            19.967891386

In [47]:
# Cell 10: Check Trade Feasibility & Identify Conflicts

import pandas as pd
import numpy as np
from decimal import Decimal

# --- Configuration ---
QUOTE_ASSET = 'USDT'

# --- Check Prerequisites ---
if 'trades_df' not in locals():
     # If trades_df is empty or doesn't exist, there's nothing to check
     print("\n'trades_df' not found or empty. Skipping feasibility check.")
     # Ensure feasible_trades_df exists and is empty for subsequent steps
     feasible_trades_df = pd.DataFrame()
elif 'portfolio_state' not in locals() or 'balances' not in portfolio_state or 'open_orders' not in portfolio_state:
     raise ValueError("Portfolio state ('portfolio_state' with 'balances' and 'open_orders') not found. Run Cell 2.")
else:
    print("\n--- Checking Trade Feasibility Against Free Balances & Open Orders ---")

    # Make a copy to add feasibility columns
    feasible_trades_df = trades_df.copy()

    # Add new columns to track status
    feasible_trades_df['Feasible'] = True # Assume feasible initially
    feasible_trades_df['ConflictType'] = '' # e.g., 'Insufficient Free Balance', 'Insufficient Free USDT'
    feasible_trades_df['BlockingOrderIDs'] = [[] for _ in range(len(feasible_trades_df))] # List to hold conflicting order IDs

    balances = portfolio_state['balances'] # DataFrame with 'Free', 'Locked' (Index: Asset)
    open_orders = portfolio_state['open_orders'] # DataFrame of open orders

    # Get free balance of the quote asset (USDT) for checking BUYs
    free_usdt = balances.loc[QUOTE_ASSET, 'Free'] if QUOTE_ASSET in balances.index else Decimal('0.0')
    print(f"Available Free {QUOTE_ASSET}: {free_usdt:.8f}")

    # Iterate through the proposed trades
    for index, trade in feasible_trades_df.iterrows():
        asset = trade['Asset']
        action = trade['Action']
        ideal_qty = trade['IdealQuantity'] # This is Decimal
        ideal_value = trade['IdealValueUSDT'] # This is Decimal

        # --- Check SELL Feasibility ---
        if action == 'SELL':
            # Check if we hold this asset and get its free balance
            if asset in balances.index:
                free_balance = balances.loc[asset, 'Free']
                # Compare ideal quantity to sell with free balance
                if ideal_qty > free_balance:
                    feasible_trades_df.loc[index, 'Feasible'] = False
                    feasible_trades_df.loc[index, 'ConflictType'] = 'Insufficient Free Balance'
                    # Find open SELL orders for this asset that are locking the balance
                    blocking_orders = open_orders[
                        (open_orders['symbol'] == f"{asset}{QUOTE_ASSET}") &
                        (open_orders['side'] == 'SELL') &
                        (open_orders['status'].isin(['NEW', 'PARTIALLY_FILLED'])) # Consider active statuses
                    ]['orderId'].tolist()
                    feasible_trades_df.at[index, 'BlockingOrderIDs'] = blocking_orders # Use .at for list assignment
            else:
                # Trying to sell an asset we don't have at all (should be rare if logic is correct)
                feasible_trades_df.loc[index, 'Feasible'] = False
                feasible_trades_df.loc[index, 'ConflictType'] = 'Asset Not Held'

        # --- Check BUY Feasibility ---
        elif action == 'BUY':
            # Compare ideal value to buy with free USDT
            if ideal_value > free_usdt:
                feasible_trades_df.loc[index, 'Feasible'] = False
                feasible_trades_df.loc[index, 'ConflictType'] = f'Insufficient Free {QUOTE_ASSET}'
                # Find open BUY orders that might be locking USDT
                # Note: Estimating locked USDT by open BUY orders is complex as it depends on price.
                # We'll just list *all* open BUY orders as potentially contributing.
                blocking_orders = open_orders[
                    (open_orders['side'] == 'BUY') &
                    (open_orders['status'].isin(['NEW', 'PARTIALLY_FILLED']))
                ]['orderId'].tolist()
                feasible_trades_df.at[index, 'BlockingOrderIDs'] = blocking_orders # Use .at for list assignment


    # --- Report Summary ---
    feasible_count = feasible_trades_df['Feasible'].sum()
    conflicting_count = len(feasible_trades_df) - feasible_count

    print(f"\nFeasibility Check Summary:")
    print(f"  {feasible_count} trade(s) appear feasible with current free balances.")
    print(f"  {conflicting_count} trade(s) have conflicts (insufficient free balance/USDT).")

    if conflicting_count > 0:
        print("\n--- Conflicting Trades Details ---")
        conflict_cols = ['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT', 'ConflictType', 'BlockingOrderIDs']
        # Ensure columns exist before selecting
        conflict_cols = [c for c in conflict_cols if c in feasible_trades_df.columns]
        with pd.option_context('display.max_colwidth', 100, 'display.float_format', '{:.8f}'.format): # Show more of list/quantity
             print(feasible_trades_df[~feasible_trades_df['Feasible']][conflict_cols])

    # Result: 'feasible_trades_df' DataFrame contains original trades plus feasibility status and conflict info.
    print(f"\n✅ Feasibility check complete. Results stored in 'feasible_trades_df'.")

# Ensure dataframe exists even if skipped
if 'feasible_trades_df' not in locals():
     feasible_trades_df = pd.DataFrame()


--- Checking Trade Feasibility Against Free Balances & Open Orders ---
Available Free USDT: 0.10189178

Feasibility Check Summary:
  0 trade(s) appear feasible with current free balances.
  9 trade(s) have conflicts (insufficient free balance/USDT).

--- Conflicting Trades Details ---
  Action Asset    IdealQuantity                  IdealValueUSDT  \
0   SELL   BTC   0.000033521895   2.611459875111559997139675196   
1   SELL  USDT  19.967891386760            19.96789138676000000   
2    BUY   ADA   1.717182474100  0.9913294422981006875462582266   
3    BUY  DOGE   5.471361688923  0.8047278772068667656359017231   
4    BUY   ETH   0.003317110798   4.954104976210199004644967914   
5    BUY  HBAR   5.967747556833  0.9196298985080235823115699750   
6    BUY   LTC   0.010236900293  0.7169924965449780674539101721   
7    BUY   SOL   0.028004185228   2.927557523764871950244568291   
8    BUY   XRP   5.698647814710   10.63139736312359179709797877   

                ConflictType              

In [48]:
# Cell 11: Enhanced LLM Assessment with Feasibility Context

import google.generativeai as genai
import pandas as pd
import os
import json
from dotenv import load_dotenv
import numpy as np
from decimal import Decimal

# --- Configuration ---
LLM_MODEL_NAME = "gemini-1.5-flash-latest" # Or your preferred Gemini model
QUOTE_ASSET = 'USDT' # Should match previous cells
RSI_PERIOD = 14 # Ensure this matches the RSI period used in Cell 5/8 for key lookup

# --- Load Environment Variables & Configure Google AI ---
print("\n--- Configuring LLM (Google AI) ---")
load_dotenv()
google_api_key = os.environ.get('GOOGLE_API_KEY')
genai_configured = False
if not google_api_key:
    print("⚠️ Warning: GOOGLE_API_KEY not found. LLM assessment will be skipped.")
else:
    try:
        genai.configure(api_key=google_api_key)
        genai_configured = True
        print("✅ Google AI configured.")
    except Exception as e:
        print(f"❌ Error configuring Google AI: {e}")

# --- Check Prerequisites ---
if not genai_configured:
     print("LLM not configured, skipping assessment.")
     # If LLM is skipped, create an empty DataFrame to avoid errors later
     llm_recommendations_df = pd.DataFrame()
elif 'feasible_trades_df' not in locals() or feasible_trades_df.empty:
    print("\nNo feasible/conflicting trades found in 'feasible_trades_df' to assess. Skipping LLM.")
    llm_recommendations_df = pd.DataFrame() # Ensure variable exists and is empty
elif 'full_market_state' not in locals() or not full_market_state:
     raise ValueError("Market state analysis ('full_market_state') not found or empty. Run Cell 8.")
else:
    print(f"\n--- Performing Enhanced LLM Assessment ({LLM_MODEL_NAME}) ---")

    # --- LLM Interaction Function (Modified) ---
    def get_llm_trade_assessment_enhanced(trade_info, symbol_state):
        """
        Generates a prompt including feasibility and asks LLM for nuanced recommendation.
        """
        if not genai_configured: return "LLM_SKIP", "LLM Not Configured"
        if not symbol_state or symbol_state.get('error'): return "ERROR", f"Invalid Symbol State Input: {symbol_state.get('error', 'Unknown error')}"

        # --- Extract data safely from inputs ---
        action = trade_info.get('Action', 'N/A')
        asset = trade_info.get('Asset', 'N/A')
        symbol = f"{asset}{QUOTE_ASSET}"
        ideal_value_str = f"{trade_info.get('IdealValueUSDT', Decimal('0')):,.2f}" # Format value nicely
        ideal_qty_str = f"{trade_info.get('IdealQuantity', Decimal('0')):.8f}" # Show quantity precisely

        # Feasibility Info
        is_feasible = trade_info.get('Feasible', False)
        conflict_type = trade_info.get('ConflictType', '')
        blocking_orders = trade_info.get('BlockingOrderIDs', [])

        # TA Info - with safe defaults and formatting
        last_close = symbol_state.get('last_close') # Should be Decimal
        last_close_str = f"{last_close:.4f}" if last_close is not None else "N/A"

        daily_trend_val = symbol_state.get('daily_trend_up')
        daily_trend_str = 'UP' if daily_trend_val is True else 'DOWN/SIDEWAYS' if daily_trend_val is False else 'Unknown'

        hourly_below_val = symbol_state.get('hourly_below_sma')
        hourly_pos_str = 'BELOW SMA20 (Dip?)' if hourly_below_val is True else 'ABOVE SMA20' if hourly_below_val is False else 'Unknown'

        rsi_key = f'last_5m_RSI_{RSI_PERIOD}' # Construct the key dynamically
        last_rsi_val = symbol_state.get(rsi_key) # Should be Decimal
        last_rsi_str = f"{last_rsi_val:.2f}" if last_rsi_val is not None else "N/A"

        rsi_oversold_val = symbol_state.get('5m_rsi_oversold')
        rsi_state_str = 'OVERSOLD (<30)' if rsi_oversold_val is True else 'NOT OVERSOLD' if rsi_oversold_val is False else 'Unknown'

        # S/R Info - format Decimals if they exist
        daily_sup = symbol_state.get('daily_support'); daily_res = symbol_state.get('daily_resistance')
        hourly_sup = symbol_state.get('hourly_support'); hourly_res = symbol_state.get('hourly_resistance')
        daily_sup_str = f"{daily_sup:.4f}" if daily_sup is not None else "N/A"
        daily_res_str = f"{daily_res:.4f}" if daily_res is not None else "N/A"
        hourly_sup_str = f"{hourly_sup:.4f}" if hourly_sup is not None else "N/A"
        hourly_res_str = f"{hourly_res:.4f}" if hourly_res is not None else "N/A"

        near_support_val = symbol_state.get('near_hourly_support')
        near_support_str = 'YES' if near_support_val is True else 'NO' if near_support_val is False else 'Unknown'

        # --- Construct Feasibility Statement ---
        if is_feasible:
            feasibility_str = f"This {action} trade appears **FEASIBLE** based on current free balances."
        else:
            feasibility_str = f"This {action} trade is **CONFLICTING**: {conflict_type}."
            if blocking_orders:
                feasibility_str += f" Potentially blocked by open order(s): {blocking_orders}."

        # --- Construct Prompt ---
        prompt = f"""
        Analyze the following cryptocurrency market state and proposed rebalancing trade for {symbol}.
        My goal is long-term portfolio growth via dynamic allocation. I prefer maker orders and buying dips.

        PROPOSED TRADE:
        * Action: {action}
        * Asset: {asset}
        * Ideal Quantity: {ideal_qty_str}
        * Ideal Value: approx ${ideal_value_str} {QUOTE_ASSET}
        * Feasibility: {feasibility_str}

        CURRENT TECHNICAL STATE ({symbol}):
        * Last Hourly Close: {last_close_str} {QUOTE_ASSET}
        * Daily Trend (vs SMA50): {daily_trend_str}
        * Hourly Position (vs SMA20): {hourly_pos_str}
        * 5min RSI ({RSI_PERIOD}) State: {rsi_state_str} (Value: {last_rsi_str})
        * Recent Daily Support / Resistance: {daily_sup_str} / {daily_res_str}
        * Recent Hourly Support / Resistance: {hourly_sup_str} / {hourly_res_str}
        * Price Near Hourly Support (within {NEAR_SUPPORT_PROXIMITY_PCT}%): {near_support_str}

        ASSESSMENT TASK:
        Based *only* on the technical state and feasibility provided:
        1. Assess the technical picture (e.g., Is it a good dip buy? Is downside likely? Is there reason to delay a sell?).
        2. Consider the feasibility/conflict information.
        3. Provide a brief rationale (1-2 sentences).
        4. Conclude with ONE of the following specific recommendations:
           - PROCEED (If technically reasonable AND feasible)
           - HOLD (If technically unclear/risky, or if waiting seems prudent)
           - MODIFY_SIZE (If technically okay but maybe quantity is too large, or feasible amount is smaller)
           - CANCEL_BLOCKERS (If technically okay but blocked by specific orders you might want to cancel)
           - SKIP (If technically poor setup or unresolvable conflict)

        Example Output:
        Rationale: [Your brief analysis considering TA and feasibility]. Recommendation: PROCEED
        """

        # --- Call LLM API ---
        try:
            # print(f"    Sending prompt to Gemini for {symbol} {action}...") # Uncomment for debug
            model = genai.GenerativeModel(LLM_MODEL_NAME)
            # Configure safety settings to be less restrictive if needed, use with caution
            safety_settings=[ {"category": c, "threshold": "BLOCK_NONE"} for c in [
                "HARM_CATEGORY_HARASSMENT", "HARM_CATEGORY_HATE_SPEECH",
                "HARM_CATEGORY_SEXUALLY_EXPLICIT", "HARM_CATEGORY_DANGEROUS_CONTENT", ] ]

            response = model.generate_content(prompt, safety_settings=safety_settings)

            # Handle potential blocks or empty responses
            if not response.candidates or (response.prompt_feedback and response.prompt_feedback.block_reason):
                reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Unknown"
                print(f"    LLM WARNING for {symbol}: Prompt/Response BLOCKED: {reason}")
                return "ERROR", f"LLM Response Blocked: {reason}"

            raw_text = response.text.strip()
            # print(f"    LLM Raw Response: {raw_text}") # Uncomment for debug

            # --- Parse Recommendation ---
            # Look for the recommendation keyword at the end or after "Recommendation:"
            recommendation = "HOLD" # Default if parsing fails
            keywords = ["PROCEED", "HOLD", "MODIFY_SIZE", "CANCEL_BLOCKERS", "SKIP"]
            lower_text = raw_text.lower()
            # Find the last occurrence of "recommendation:"
            rec_marker = lower_text.rfind("recommendation:")
            search_area = lower_text[rec_marker:] if rec_marker != -1 else lower_text

            found_rec = None
            # Check from longest keyword to shortest to avoid partial matches if possible
            for keyword in sorted(keywords, key=len, reverse=True):
                if keyword.lower() in search_area.split()[-2:]: # Check last few words
                    found_rec = keyword
                    break
            if found_rec:
                 recommendation = found_rec

            return recommendation, raw_text # Return parsed recommendation and full text

        except Exception as e:
            print(f"    ❌ Error calling/parsing LLM for {symbol}: {e}")
            return "ERROR", f"LLM API Error: {e}"

    # --- Iterate Through Trades and Get Assessments ---
    llm_results = []
    print(f"Assessing {len(feasible_trades_df)} trade proposals...")
    for index, trade in feasible_trades_df.iterrows():
        symbol_ws = f"{trade['Asset']}{QUOTE_ASSET}"
        asset_state = full_market_state.get(symbol_ws) # Get TA state

        print(f"  Assessing {trade['Action']} {trade['Asset']}...")

        if not asset_state:
             print(f"    Skipping LLM: Market state not found for {symbol_ws}.")
             rec = "SKIP"
             reason = "Missing market state data"
        elif asset_state.get('error'):
             print(f"    Skipping LLM: Error in market state for {symbol_ws}: {asset_state['error']}")
             rec = "SKIP"
             reason = f"Market state error: {asset_state['error']}"
        else:
            # Call the enhanced function
            rec, reason = get_llm_trade_assessment_enhanced(trade.to_dict(), asset_state)
            print(f"    LLM Recommendation: {rec}")
            # if rec == 'ERROR': print(f"      LLM Reason/Error: {reason}") # Uncomment for details on errors

        llm_results.append({
            'LLM_Recommendation': rec,
            'LLM_Rationale': reason
        })

    # --- Combine Recommendations with Trades DataFrame ---
    llm_df = pd.DataFrame(llm_results, index=feasible_trades_df.index)
    llm_recommendations_df = feasible_trades_df.join(llm_df)

    print("\n--- LLM Assessment Summary ---")
    print(llm_recommendations_df[['Action', 'Asset', 'IdealValueUSDT', 'Feasible', 'ConflictType', 'LLM_Recommendation']].round(2))
    print("\n✅ LLM assessment complete. Results stored in 'llm_recommendations_df'.")


# Ensure the final DataFrame variable exists even if LLM was skipped
if 'llm_recommendations_df' not in locals():
     llm_recommendations_df = pd.DataFrame()
     print("\nLLM assessment was skipped. 'llm_recommendations_df' is empty.")


--- Configuring LLM (Google AI) ---
✅ Google AI configured.

--- Performing Enhanced LLM Assessment (gemini-1.5-flash-latest) ---
Assessing 9 trade proposals...
  Assessing SELL BTC...
    LLM Recommendation: SKIP
  Assessing SELL USDT...
    Skipping LLM: Market state not found for USDTUSDT.
  Assessing BUY ADA...
    LLM Recommendation: CANCEL_BLOCKERS
  Assessing BUY DOGE...
    LLM Recommendation: CANCEL_BLOCKERS
  Assessing BUY ETH...
    LLM Recommendation: SKIP
  Assessing BUY HBAR...
    LLM Recommendation: SKIP
  Assessing BUY LTC...
    LLM Recommendation: CANCEL_BLOCKERS
  Assessing BUY SOL...
    LLM Recommendation: CANCEL_BLOCKERS
  Assessing BUY XRP...
    LLM Recommendation: CANCEL_BLOCKERS

--- LLM Assessment Summary ---
  Action Asset                  IdealValueUSDT  Feasible  \
0   SELL   BTC   2.611459875111559997139675196     False   
1   SELL  USDT            19.96789138676000000     False   
2    BUY   ADA  0.9913294422981006875462582266     False   
3    BUY  DO

In [49]:
# Cell 12: Filter for Proceedable Trades & Final Check

import pandas as pd
from decimal import Decimal # Import if needed for type checking

# --- Check Prerequisites ---
if 'llm_recommendations_df' not in locals():
    print("⚠️ 'llm_recommendations_df' not found. Skipping filtering.")
    # Ensure final_trades_to_format exists and is empty
    final_trades_to_format_df = pd.DataFrame()
elif llm_recommendations_df.empty:
    print("ℹ️ 'llm_recommendations_df' is empty. No trades to filter.")
    final_trades_to_format_df = pd.DataFrame()
else:
    print("\n--- Filtering for LLM 'PROCEED' Recommendations on Feasible Trades ---")

    # Filter conditions:
    # 1. LLM_Recommendation must be 'PROCEED'
    # 2. Feasible flag must be True (already checked against free balance)
    proceed_condition = (llm_recommendations_df['LLM_Recommendation'] == 'PROCEED')
    feasible_condition = (llm_recommendations_df['Feasible'] == True)

    final_trades_to_format_df = llm_recommendations_df[proceed_condition & feasible_condition].copy()

    # --- Report Summary ---
    original_count = len(llm_recommendations_df)
    proceed_count = len(final_trades_to_format_df)
    hold_skip_count = original_count - proceed_count

    print(f"Out of {original_count} assessed trades:")
    print(f"  - {proceed_count} trade(s) marked 'PROCEED' by LLM and are feasible.")
    if hold_skip_count > 0:
        print(f"  - {hold_skip_count} trade(s) filtered out (HOLD, SKIP, MODIFY, CANCEL_BLOCKERS, or Not Feasible).")
        # Optional: Show trades not proceeding
        # print("\n--- Trades NOT Proceeding to Formatting ---")
        # cols = ['Action', 'Asset', 'IdealValueUSDT', 'Feasible', 'ConflictType', 'LLM_Recommendation']
        # cols = [c for c in cols if c in llm_recommendations_df.columns]
        # with pd.option_context('display.float_format', '{:.4f}'.format):
        #     print(llm_recommendations_df[~(proceed_condition & feasible_condition)][cols])


    if not final_trades_to_format_df.empty:
        print("\n--- Trades Ready for Formatting ---")
        # Display the trades that will proceed
        display_cols = ['Action', 'Asset', 'IdealQuantity', 'IdealValueUSDT', 'CurrentPrice', 'LLM_Recommendation']
         # Ensure columns exist before selecting
        display_cols = [c for c in display_cols if c in final_trades_to_format_df.columns]
        with pd.option_context('display.float_format', '{:.8f}'.format):
            print(final_trades_to_format_df[display_cols])
        print("\n✅ Filtering complete. Proceedable trades stored in 'final_trades_to_format_df'.")
    else:
        print("\n✅ Filtering complete. No trades marked 'PROCEED' and feasible found.")


# Ensure the final DataFrame exists, even if empty
if 'final_trades_to_format_df' not in locals():
     final_trades_to_format_df = pd.DataFrame()


--- Filtering for LLM 'PROCEED' Recommendations on Feasible Trades ---
Out of 9 assessed trades:
  - 0 trade(s) marked 'PROCEED' by LLM and are feasible.
  - 9 trade(s) filtered out (HOLD, SKIP, MODIFY, CANCEL_BLOCKERS, or Not Feasible).

✅ Filtering complete. No trades marked 'PROCEED' and feasible found.


In [51]:
# Cell 13: Format Proceedable Trades as LIMIT Orders (Maker Strategy)

import pandas as pd
import os
import numpy as np
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN # Import Decimal for precision handling
import json

# --- Configuration ---
QUOTE_ASSET = 'USDT'
# Maker strategy: Place BUY orders at the best bid, SELL orders at the best ask
# This aims to *add* liquidity and potentially get maker fees.
PLACE_BUY_AT = 'BID' # Or 'ASK'
PLACE_SELL_AT = 'ASK' # Or 'BID'

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
    raise RuntimeError("Binance client not initialized. Run Cell 1.")
if 'format_price_correctly' not in locals() or 'adjust_quantity_to_step' not in locals():
     raise RuntimeError("Formatting helper functions not defined. Run Cell 1.")
if 'final_trades_to_format_df' not in locals():
    print("⚠️ 'final_trades_to_format_df' not found. Skipping order formatting.")
    final_limit_orders_to_test = [] # Ensure list exists
elif final_trades_to_format_df.empty:
    print("ℹ️ 'final_trades_to_format_df' is empty. No trades to format.")
    final_limit_orders_to_test = [] # Ensure list exists
else:
    print("\n--- Formatting Proceedable Trades into LIMIT Orders (Maker Strategy) ---")
    print(f"BUY orders target: {PLACE_BUY_AT}, SELL orders target: {PLACE_SELL_AT}")

    # --- Get Exchange Info ---
    print("Fetching exchange information...")
    symbols_info = None
    try:
        exchange_info = client.get_exchange_info()
        symbols_info = {item['symbol']: item for item in exchange_info['symbols']}
        print("✅ Exchange information received.")
    except Exception as e:
        print(f"❌ Error fetching exchange info: {e}. Cannot format orders.")

    final_limit_orders_to_test = [] # List to hold final order parameter dictionaries

    if symbols_info: # Proceed only if we have exchange rules
        # Iterate through the DataFrame of trades ready for formatting
        for index, trade in final_trades_to_format_df.iterrows():
            action = trade['Action'] # 'BUY' or 'SELL'
            asset = trade['Asset']
            # Use the IdealQuantity calculated earlier (should be Decimal)
            quantity_ideal = trade['IdealQuantity']

            if asset == QUOTE_ASSET: continue # Skip trades involving only the quote asset

            symbol = f"{asset}{QUOTE_ASSET}"
            print(f"\nProcessing Trade {index+1}: {action} {asset} ({symbol})")
            print(f"  Ideal Quantity (approx): {quantity_ideal:.8f}")

            if symbol not in symbols_info:
                print(f"  ⚠️ Warning: Symbol {symbol} not found in exchange info. Skipping.")
                continue

            symbol_info = symbols_info[symbol]
            filters = {f['filterType']: f for f in symbol_info['filters']}

            # --- Get Order Book for Maker Price ---
            best_bid = None
            best_ask = None
            try:
                 depth = client.get_order_book(symbol=symbol, limit=5)
                 if depth['bids'] and depth['asks']:
                      best_bid = Decimal(depth['bids'][0][0])
                      best_ask = Decimal(depth['asks'][0][0])
                      # print(f"  Best Bid: {best_bid}, Best Ask: {best_ask}") # Uncomment for debug
                 else:
                      print(f"  ⚠️ Warning: Order book empty/invalid for {symbol}. Skipping.")
                      continue
            except Exception as e:
                 print(f"  ⚠️ Warning: Could not fetch order book for {symbol}: {e}. Skipping.")
                 continue

            # --- Determine Target Limit Price (Raw Decimal) based on strategy ---
            if action == 'BUY':
                 limit_price_raw = best_bid if PLACE_BUY_AT == 'BID' else best_ask
                 print(f"  Targeting BUY Limit Price at {PLACE_BUY_AT}: {limit_price_raw}")
            elif action == 'SELL':
                 limit_price_raw = best_ask if PLACE_SELL_AT == 'ASK' else best_bid
                 print(f"  Targeting SELL Limit Price at {PLACE_SELL_AT}: {limit_price_raw}")
            else: # Should not happen
                 print(f"  ⚠️ Unknown action '{action}'. Skipping.")
                 continue


            # --- Get Filters & Adjust Price/Qty using Helper Functions ---
            price_filter = filters.get('PRICE_FILTER', {})
            lot_size_filter = filters.get('LOT_SIZE', {})
            # Use NOTIONAL first, fallback to MIN_NOTIONAL
            notional_filter = filters.get('NOTIONAL', {})
            min_notional_str = notional_filter.get('minNotional')
            if min_notional_str is None:
                min_notional_filter = filters.get('MIN_NOTIONAL', {})
                min_notional_str = min_notional_filter.get('minNotional', '0.0')

            tick_size = price_filter.get('tickSize')
            step_size = lot_size_filter.get('stepSize')

            if not tick_size or not step_size or min_notional_str is None:
                print(f"  ⚠️ Warning: Missing critical filter info for {symbol}. Skipping.")
                continue

            try:
                 min_notional = Decimal(min_notional_str)
            except Exception as dec_err:
                 print(f"  ⚠️ Warning: Invalid minNotional '{min_notional_str}': {dec_err}. Skipping.")
                 continue

            # Apply formatting using Helper Functions from Cell 1
            # These MUST return strings formatted to the exact required precision
            try:
                # Price formatting (e.g., using tickSize)
                adjusted_limit_price_str = format_price_correctly(limit_price_raw, tick_size)
                # Quantity formatting (using stepSize)
                adjusted_quantity_str = adjust_quantity_to_step(quantity_ideal, step_size)

                # Convert back to Decimal ONLY for the notional value check
                adjusted_limit_price_dec = Decimal(adjusted_limit_price_str)
                adjusted_quantity_dec = Decimal(adjusted_quantity_str)

                print(f"  Formatted Price Str: {adjusted_limit_price_str}")
                print(f"  Adjusted Quantity Str: {adjusted_quantity_str}")

            except Exception as fmt_err:
                 print(f"  ❌ Error during price/quantity formatting: {fmt_err}. Skipping.")
                 continue

            # --- Check Adjusted Quantity & MIN_NOTIONAL ---
            if adjusted_quantity_dec <= 0:
                 print(f"  ⚠️ Adjusted quantity ({adjusted_quantity_dec}) is zero or less after applying step size. Skipping.")
                 continue

            order_value = adjusted_quantity_dec * adjusted_limit_price_dec

            if min_notional > 0 and order_value < min_notional:
                 print(f"  ⚠️ Order value ({order_value:.8f}) is below minimum notional ({min_notional:.8f}). Skipping.")
                 continue

            print(f"  ✅ Order meets minimum notional ({order_value:.8f} >= {min_notional:.8f}).")

            # --- Add Validated Order Parameters to List ---
            final_limit_orders_to_test.append({
                'symbol': symbol,
                'side': action.upper(), # Ensure 'BUY' or 'SELL'
                'type': 'LIMIT',
                'timeInForce': 'GTC', # Good Till Cancelled
                'quantity': adjusted_quantity_str, # Pass the FORMATTED STRING
                'price': adjusted_limit_price_str   # Pass the FORMATTED STRING
            })
            print(f"  ✅ Added valid LIMIT order parameters for {symbol} to list.")
        # End of loop through trades
    else:
        # This case handles if symbols_info was None due to API error
        print("Skipping order formatting because exchange info could not be fetched.")


# --- Display Final Orders Prepared for Testing ---
print("\n--- Final LIMIT Orders Ready for Testing ---")
if not final_limit_orders_to_test:
    print("No trades were formatted (list is empty).")
else:
    # Create DataFrame for display
    final_orders_preview_df = pd.DataFrame(final_limit_orders_to_test)
    # Display relevant columns
    print(final_orders_preview_df[['symbol', 'side', 'type', 'timeInForce', 'quantity', 'price']])
    print(f"\nStored {len(final_limit_orders_to_test)} order(s) in 'final_limit_orders_to_test'.")


# Ensure list exists, even if empty
if 'final_limit_orders_to_test' not in locals():
     final_limit_orders_to_test = []

ℹ️ 'final_trades_to_format_df' is empty. No trades to format.

--- Final LIMIT Orders Ready for Testing ---
No trades were formatted (list is empty).


In [52]:
# Cell 14: Test Final LIMIT Orders (Simulation)

import pandas as pd # Only needed if displaying results from non-empty list

# --- Check Prerequisites ---
if 'client' not in locals() or client is None:
     raise RuntimeError("Binance client not initialized. Run Cell 1.")
if 'final_limit_orders_to_test' not in locals():
     print("⚠️ 'final_limit_orders_to_test' list not found. Cannot test orders.")
elif not final_limit_orders_to_test: # Check if the list is empty
     print("ℹ️ No formatted orders in 'final_limit_orders_to_test'. Skipping test orders.")
else:
    # This block will only execute if final_limit_orders_to_test is NOT empty
    print("\n--- Sending FINAL LIMIT Test Orders (Simulation - No Real Trades) ---")

    successful_tests = 0
    failed_tests = 0
    test_results = [] # Store results for review

    for order_params in final_limit_orders_to_test:
        print(f"Attempting TEST {order_params['side']} for {order_params['quantity']} {order_params['symbol']} at price {order_params['price']}...")
        result = {}
        try:
            # create_test_order should return an empty dict {} on success
            test_result = client.create_test_order(
                symbol=order_params['symbol'],
                side=order_params['side'],
                type=order_params['type'],
                timeInForce=order_params['timeInForce'],
                quantity=order_params['quantity'], # Pass the formatted string
                price=order_params['price']         # Pass the formatted string
            )
            # Binance test order success is indicated by an empty dictionary response
            if isinstance(test_result, dict) and not test_result:
                print(f"  ✅ SUCCESS: Test order for {order_params['symbol']} validated.")
                result = {**order_params, 'test_status': 'Success', 'error': None}
                successful_tests += 1
            else:
                # Should not happen on successful test, indicates unexpected API behavior
                print(f"  ⚠️ UNEXPECTED RESULT (Expected empty dict): {test_result}")
                result = {**order_params, 'test_status': 'Failed', 'error': f'Unexpected Result: {test_result}'}
                failed_tests += 1

        except Exception as e:
            print(f"  ❌ FAILED: Test order for {order_params['symbol']} failed validation.")
            print(f"     Error: {e}")
            result = {**order_params, 'test_status': 'Failed', 'error': str(e)}
            failed_tests += 1
        test_results.append(result)

    print("\n--- FINAL LIMIT Test Order Summary ---")
    print(f"Attempted Tests: {len(final_limit_orders_to_test)}")
    print(f"Successful Validations: {successful_tests}")
    print(f"Failed Validations    : {failed_tests}")

    if failed_tests > 0:
        print("\n--- Failed Test Order Details ---")
        failed_df = pd.DataFrame([r for r in test_results if r['test_status'] == 'Failed'])
        print(failed_df[['symbol', 'side', 'quantity', 'price', 'error']])
        print("\nReview failed test orders before proceeding to real execution.")

ℹ️ No formatted orders in 'final_limit_orders_to_test'. Skipping test orders.


In [53]:
# Cell 15: Execute REAL Formatted Orders (CAUTION!)

import pandas as pd
import json
import time # For potential delays or confirmation pauses

# --- Configuration ---
EXECUTE_REAL_ORDERS = True # <<< MASTER SAFETY SWITCH. Set to False to prevent execution.
REQUIRE_CONFIRMATION = True # Set to True to require manual 'yes' input before executing.

# --- Check Prerequisites ---
if not EXECUTE_REAL_ORDERS:
    print("--- REAL ORDER EXECUTION IS DISABLED (EXECUTE_REAL_ORDERS = False) ---")
    executed_orders_info = [] # Ensure list exists
    failed_orders_exec = [] # Ensure list exists
elif 'client' not in locals() or client is None:
     raise RuntimeError("Binance client not initialized. Run Cell 1.")
elif 'final_limit_orders_to_test' not in locals():
     print("⚠️ 'final_limit_orders_to_test' list not found. Cannot execute.")
     executed_orders_info = []
     failed_orders_exec = []
elif not final_limit_orders_to_test: # Check if the list is empty
     print("ℹ️ No formatted orders passed testing. Skipping real order execution.")
     executed_orders_info = []
     failed_orders_exec = []
else:
    # --- Safety Confirmation ---
    print("\n" + "="*50)
    print(" !!! WARNING: ABOUT TO EXECUTE REAL ORDERS !!!")
    print("="*50)
    print("The following orders passed formatting and testing:")
    orders_to_exec_df = pd.DataFrame(final_limit_orders_to_test)
    print(orders_to_exec_df[['symbol', 'side', 'type', 'quantity', 'price']])
    print(f"\nTotal orders to execute: {len(orders_to_exec_df)}")
    print("="*50)

    proceed = False
    if REQUIRE_CONFIRMATION:
        try:
            # Use input() which works in notebooks
            confirm = input("Type 'yes' to confirm and execute these orders: ")
            if confirm.lower() == 'yes':
                proceed = True
                print("Confirmation received. Proceeding with execution...")
            else:
                print("Confirmation not received. EXECUTION CANCELLED.")
        except EOFError:
             # Handle environments where input() might not work (e.g., non-interactive execution)
             print("Could not get confirmation (EOFError). EXECUTION CANCELLED.")
    else:
        print("Skipping manual confirmation (REQUIRE_CONFIRMATION = False).")
        proceed = True # Proceed without manual input if flag is False

    # --- Execute Orders if Confirmed ---
    executed_orders_info = [] # List to store results of successful orders
    failed_orders_exec = []   # List to store parameters and errors of failed orders

    if proceed:
        print("\n--- Sending REAL LIMIT Orders ---")
        for order_params in final_limit_orders_to_test:
            print(f"\nAttempting REAL {order_params['side']} {order_params['type']} for {order_params['quantity']} {order_params['symbol']} at {order_params['price']}...")
            try:
                # --- THE ACTUAL LIVE ORDER PLACEMENT ---
                order_result = client.create_order(
                    symbol=order_params['symbol'],
                    side=order_params['side'],
                    type=order_params['type'],      # LIMIT
                    timeInForce=order_params['timeInForce'], # GTC
                    quantity=order_params['quantity'], # Formatted string
                    price=order_params['price']      # Formatted string
                )
                # --- ORDER PLACED SUCCESSFULLY ---

                print(f"  ✅ SUCCESS: Order for {order_params['symbol']} PLACED.")
                print(f"     Order ID: {order_result.get('orderId', 'N/A')}")
                # print("     Full Result:") # Uncomment for more detail
                # print(json.dumps(order_result, indent=4)) # Pretty print result
                executed_orders_info.append(order_result)
                time.sleep(0.5) # Small delay between orders to avoid rate limits

            except Exception as e:
                print(f"  ❌ FAILED: Order placement for {order_params['symbol']} failed.")
                print(f"     Error: {e}")
                failed_orders_exec.append({**order_params, 'error': str(e)})

        # --- Execution Summary ---
        print("\n--- Real Order Execution Summary ---")
        print(f"Successfully placed orders: {len(executed_orders_info)}")
        print(f"Failed orders           : {len(failed_orders_exec)}")

        if failed_orders_exec:
            print("\n--- Failed Order Details ---")
            failed_exec_df = pd.DataFrame(failed_orders_exec)
            print(failed_exec_df[['symbol', 'side', 'quantity', 'price', 'error']])
            print("\nReview failed orders and account status.")
        elif executed_orders_info:
             print("\nCheck order status using Cell 2 (Get State) or the exchange interface.")

    # End of 'if proceed:' block
    elif not proceed and REQUIRE_CONFIRMATION:
         # This block executes if confirmation was required but not given
         print("Execution skipped due to lack of confirmation.")


# Ensure lists exist for potential use later, even if execution was skipped/failed
if 'executed_orders_info' not in locals(): executed_orders_info = []
if 'failed_orders_exec' not in locals(): failed_orders_exec = []

ℹ️ No formatted orders passed testing. Skipping real order execution.


In [32]:
# Cell 17: Get Comprehensive Portfolio State (Balances & Open Orders)

import pandas as pd
import os
from binance.client import Client
from dotenv import load_dotenv
from decimal import Decimal, ROUND_DOWN # Use Decimal for precision
from datetime import datetime

# --- Configuration ---
QUOTE_ASSET = 'USDT' # Define your quote asset

# --- Load API Keys and Initialize Client ---
print("--- Initializing Client for Comprehensive State Check ---")
load_dotenv()
api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = None
if api_key and api_secret:
    try:
        # Ensure tld='us' for Binance.US
        client = Client(api_key, api_secret, tld='us')
        client.ping()
        print("✅ Binance client initialized successfully.")
    except Exception as e:
        print(f"❌ Error initializing Binance client: {e}")
        client = None
else:
    print("❌ API Key or Secret not found.")
    client = None

# --- Function to Get Balances (using Decimal) ---
def get_current_balances_detailed(api_client):
    """
    Fetches account balances from Binance.US, returns non-zero balances
    as a Pandas DataFrame with Decimal types for precision.
    """
    if not api_client:
        print("API Client is not available for get_current_balances_detailed.")
        # Return DataFrame with correct columns and types
        return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)


    try:
        print("Fetching account balances...")
        account_info = api_client.get_account()
        balances_raw = account_info.get('balances', [])

        processed_balances = []
        for asset_info in balances_raw:
            # Use Decimal for precision
            free = Decimal(asset_info['free'])
            locked = Decimal(asset_info['locked'])
            if free > 0 or locked > 0:
                processed_balances.append({
                    'Asset': asset_info['asset'],
                    'Free': free,
                    'Locked': locked
                })

        if not processed_balances:
            print("No assets with non-zero balance found.")
            return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)

        balances_df = pd.DataFrame(processed_balances)
        balances_df.set_index('Asset', inplace=True)
        print("✅ Balances fetched successfully.")
        return balances_df

    except Exception as e:
        print(f"❌ Error fetching account balances: {e}")
        return pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object)

# --- Function to Get Open Orders ---
def get_open_orders_detailed(api_client):
    """
    Fetches all open orders from Binance.US and returns them as a DataFrame.
    """
    if not api_client:
        print("API Client is not available for get_open_orders_detailed.")
        return pd.DataFrame()

    try:
        print("Fetching open orders...")
        open_orders_raw = api_client.get_open_orders()

        if not open_orders_raw:
            print("No open orders found.")
            return pd.DataFrame() # Return empty DataFrame if no orders

        print(f"Fetched {len(open_orders_raw)} open order(s).")
        open_orders_df = pd.DataFrame(open_orders_raw)

        # Convert numeric columns (use Decimal where appropriate, float otherwise)
        for col in ['price', 'origQty', 'executedQty', 'cummulativeQuoteQty', 'stopPrice']:
            if col in open_orders_df.columns:
                open_orders_df[col] = open_orders_df[col].apply(Decimal)

        # Convert timestamp columns
        for col in ['time', 'updateTime']:
            if col in open_orders_df.columns:
                open_orders_df[col] = pd.to_datetime(open_orders_df[col], unit='ms', utc=True, errors='coerce')

        print("✅ Open orders fetched and processed successfully.")
        # Sort for consistency
        return open_orders_df.sort_values(by=['symbol', 'time'], ascending=[True, False]).reset_index(drop=True)

    except Exception as e:
        print(f"❌ Error fetching or processing open orders: {e}")
        return pd.DataFrame() # Return empty DataFrame on error


# --- Main Execution Logic for this Cell ---
portfolio_state = {
    "balances": pd.DataFrame(columns=['Free', 'Locked'], index=pd.Index([], name='Asset'), dtype=object),
    "open_orders": pd.DataFrame()
}

if client:
    portfolio_state["balances"] = get_current_balances_detailed(client)
    portfolio_state["open_orders"] = get_open_orders_detailed(client)

    print("\n--- Comprehensive Portfolio State Summary ---")

    if not portfolio_state["balances"].empty:
        print("\nBalances (Free/Locked):")
        # Display with appropriate formatting
        with pd.option_context('display.float_format', '{:.8f}'.format):
             print(portfolio_state["balances"])
    else:
        print("\nBalances: Could not be fetched or none found.")

    if not portfolio_state["open_orders"].empty:
        print("\nOpen Orders:")
        display_cols = ['symbol', 'orderId', 'side', 'type', 'status', 'price', 'origQty', 'executedQty', 'time']
        display_cols = [col for col in display_cols if col in portfolio_state["open_orders"].columns]
        with pd.option_context('display.float_format', '{:.8f}'.format):
            print(portfolio_state["open_orders"][display_cols])
    else:
        print("\nOpen Orders: None found or could not be fetched.")

    print("\n--- State captured in 'portfolio_state' dictionary ---")

else:
    print("❌ Client not initialized. Cannot fetch portfolio state.")

# Result: 'portfolio_state' dictionary now holds two DataFrames:
# portfolio_state['balances']: Index=Asset, Columns=['Free', 'Locked'] (Decimal)
# portfolio_state['open_orders']: DataFrame of open orders with Decimal quantities/prices.

--- Initializing Client for Comprehensive State Check ---
✅ Binance client initialized successfully.
Fetching account balances...
✅ Balances fetched successfully.
Fetching open orders...
Fetched 5 open order(s).
✅ Open orders fetched and processed successfully.

--- Comprehensive Portfolio State Summary ---

Balances (Free/Locked):
             Free       Locked
Asset                         
BTC    0.00000710   0.00008000
USDT   0.10189178  26.60000000
BUSD   0.14956400         0E-8
WAVES  0.01000000         0E-8

Open Orders:
    symbol     orderId  side         type status           price     origQty  \
0  BTCUSDT  1466046736  SELL  LIMIT_MAKER    NEW  91000.00000000  0.00004000   
1  BTCUSDT  1466046691  SELL  LIMIT_MAKER    NEW  87000.00000000  0.00004000   
2  BTCUSDT  1466045855   BUY  LIMIT_MAKER    NEW  50000.00000000  0.00018000   
3  BTCUSDT  1466045747   BUY  LIMIT_MAKER    NEW  64000.00000000  0.00014000   
4  BTCUSDT  1466045383   BUY  LIMIT_MAKER    NEW  72000.00000000  